In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install albumentations

In [ ]:
import os
import albumentations as A
import numpy as np
import cv2
from PIL import Image
import random

base_dir = "/content/drive/MyDrive/project_lymphoma/SSL"

# Создание нужных папок
for subdir in ["images", "masks", "augmentation/images", "augmentation/masks"]:
    os.makedirs(f"{base_dir}/{subdir}", exist_ok=True)

# Получаем список изображений
image_dir = '/content/drive/MyDrive/project_lymphoma/TrainSetObjectAndBG_256x256/TrainSetObjectAndBG_256x256/background_256x256_patches/images'
mask_dir = '/content/drive/MyDrive/project_lymphoma/TrainSetObjectAndBG_256x256/TrainSetObjectAndBG_256x256/background_256x256_patches/masks'
image_files = [f for f in os.listdir(image_dir) if f.endswith(".png")]
for img_file in image_files:

    base_name = img_file.replace('.png', '')
    img_path = os.path.join(image_dir, img_file)
    mask_path = os.path.join(mask_dir, base_name+"_mask.png")
    if not os.path.exists(mask_path):
        print(f"Warning: mask for {mask_path} not found, skipping")
        continue

    # Чтение изображений и масок
    original_image = np.array(Image.open(img_path).convert("RGB"))
    original_mask = np.array(Image.open(mask_path).convert("L"))
    original_mask = 255 - original_mask

    Image.fromarray(original_image).save(f"{base_dir}/images/{base_name}.png")
    Image.fromarray(original_mask).save(f"{base_dir}/masks/{base_name}_mask.png")
    # Аугментации
    for aug_idx in range(10):
        transform = A.ReplayCompose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ColorJitter(brightness=random.uniform(0.01, 0.3), contrast=random.uniform(0.1, 0.03), saturation=random.uniform(0.01, 0.4), hue=random.uniform(0.01, 0.1), p=0.5),
            A.GaussianBlur(blur_limit=(3, 3), sigma_limit=(random.uniform(0.025, 0.1), 0.5), p=0.5),
        ])
        augmented = transform(image=original_image, mask=original_mask)
        aug_image = augmented["image"].astype(np.uint8)
        aug_mask = augmented["mask"].astype(np.uint8)

        # Инвертирование маски
        aug_mask = 255 - aug_mask

        # Названия применённых трансформаций
        replay = augmented["replay"]
        applied = [t["__class_fullname__"].split(".")[-1] for t in replay["transforms"] if t["applied"]]
        suffix = "_".join(applied) if applied else "no_aug"

        # Создание поддиректорий
        image_subdir = f"{base_dir}/augmentation/images/{base_name}"
        mask_subdir = f"{base_dir}/augmentation/masks/{base_name}"
        os.makedirs(image_subdir, exist_ok=True)
        os.makedirs(mask_subdir, exist_ok=True)
        # Сохранение
        aug_image_name = f"{base_name}_aug{aug_idx}_{suffix}.png"
        aug_mask_name = f"{base_name}_aug{aug_idx}_{suffix}_mask.png"

        Image.fromarray(aug_image).save(os.path.join(image_subdir, aug_image_name))
        Image.fromarray(aug_mask).save(os.path.join(mask_subdir, aug_mask_name))


In [ ]:
len(os.listdir('/content/drive/MyDrive/project_lymphoma/SSL/augmentation/images'))

1576